In [1]:
#Import pandas and the data
# %%
import pandas as pd

# %%
data = pd.read_csv("D:\\OneDrive\\Desktop\\Work\\python\\RBCDS\\Challenge\\Second-half\\Project_4\\Weekly_Provisional_Counts_of_Deaths_by_State_and_Select_Causes__2020-2023_20240410.csv")



In [2]:
#Find datatypes
print(data.dtypes)

Data As Of                                                                                            object
Jurisdiction of Occurrence                                                                            object
MMWR Year                                                                                              int64
MMWR Week                                                                                              int64
Week Ending Date                                                                                      object
All Cause                                                                                              int64
Natural Cause                                                                                          int64
Septicemia (A40-A41)                                                                                 float64
Malignant neoplasms (C00-C97)                                                                        float64
Diabetes mellitus (

In [3]:
#Convert Date time
data["Week Ending Date"] = pd.to_datetime(data["Week Ending Date"])
data['Year'] = data['Week Ending Date'].dt.year
data['Month'] = data['Week Ending Date'].dt.month
data['Day'] = data['Week Ending Date'].dt.day
data_agg = data.groupby(['Year', 'Month', 'Jurisdiction of Occurrence'])['All Cause'].sum().reset_index()



In [4]:
# Check for missing values
print(data_agg.isnull().sum())


Year                          0
Month                         0
Jurisdiction of Occurrence    0
All Cause                     0
dtype: int64


In [5]:
# Encode categorical variables using one-hot encoding
data_agg = pd.get_dummies(data_agg, columns=['Jurisdiction of Occurrence'])

In [6]:
#Scaling features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_agg[['Year', 'Month', 'All Cause']] = scaler.fit_transform(data_agg[['Year', 'Month', 'All Cause']])

In [7]:
#training the data 
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (y)
X = data_agg.drop('All Cause', axis=1)
y = data_agg['All Cause']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create a decision tree regression model
dt_regressor = DecisionTreeRegressor(random_state=42)

# Train the model on the training set
dt_regressor.fit(X_train, y_train)

# Evaluate the model's performance on the testing set
y_pred = dt_regressor.predict(X_test)

In [9]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)


Mean Squared Error: 0.04579609376699545
Root Mean Squared Error: 0.21400021908165293
Mean Absolute Error: 0.0533773037205495
R-squared: 0.9686374680170666


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# Define the hyperparameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Create a decision tree regression model
dt_regressor = DecisionTreeRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(dt_regressor, param_grid, cv=5, scoring='r2', return_train_score=True, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding mean cross-validated score
print('Best hyperparameters:', grid_search.best_params_)
print('Best mean cross-validated score:', grid_search.best_score_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2}
Best mean cross-validated score: 0.9697242233345076


In [11]:
# Create a decision tree regression model with the best hyperparameters
dt_regressor_best = DecisionTreeRegressor(max_depth=None, max_features='sqrt', min_samples_leaf=2, min_samples_split=2, random_state=42)

# Train the model on the training data
dt_regressor_best.fit(X_train, y_train)

# Evaluate the model on the testing data
y_pred = dt_regressor_best.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)

Mean Squared Error: 0.04652613204589213
Root Mean Squared Error: 0.21569917024850171
Mean Absolute Error: 0.05181749868494277
R-squared: 0.9681375160127067


In [12]:


# %%
data

# Selecting which State
filtered_data = data[data["Jurisdiction of Occurrence"] == "New Jersey"]
filtered_data

# %%
# filtered_data = data_agg[data_agg["Jurisdiction of Occurrence_Alabama"] == 1]
# filtered_data

# %%
import pandas as pd

# Assuming the date column is named 'Date' and is in the format 'MM/DD/YYYY'
data['Date'] = pd.to_datetime(data['Data As Of'], format='%m/%d/%Y')

# Extract year, month, and day as separate columns
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Drop the original 'Date' column if it's no longer needed
data = data.drop('Date', axis=1)


# %%
filtered_data = filtered_data.drop('Data As Of', axis=1)
filtered_data = filtered_data.drop('Jurisdiction of Occurrence', axis=1)
filtered_data = filtered_data.drop('Week Ending Date', axis=1)
filtered_data.drop(filtered_data.columns[20:36], axis=1, inplace=True)



# %%
filtered_data = filtered_data.drop('flag_allcause', axis=1)
filtered_data = filtered_data.drop('flag_natcause', axis=1)
filtered_data = filtered_data.drop('flag_sept', axis=1)

# %%
filtered_data.to_csv("filtered_data.csv", index=False)

# %%
filtered_data

# Which disease
X = filtered_data.drop('Alzheimer disease (G30)', axis=1)
y = filtered_data['Alzheimer disease (G30)']

X = X.fillna(0)
y = y.fillna(0)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:

from sklearn.tree import DecisionTreeRegressor

dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train, y_train)


# %%
y_pred = dt_regressor.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R-squared:', r2)

Mean Squared Error: 105.92307692307692
Root Mean Squared Error: 10.29189374814358
Mean Absolute Error: 8.435897435897436
R-squared: 0.6602509489666807


In [14]:

# Predict the target values for the test set (X_test)
y_pred = dt_regressor.predict(X_test)

# Print the predicted values
print("Predicted values:\n", y_pred)


# %%
import pandas as pd

# Create a new DataFrame with the actual and predicted values
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Print the first 10 rows of the predictions DataFrame
print("First 10 rows of predictions:\n", predictions_df.head(10))


# %%
predictions_df

# %%
#for loop, for each state, all diseases




Predicted values:
 [ 42. 122.  40.  55.  56.  60.  64. 122.  53.  36.  25.  81.  37.  48.
  34.  49.  51.  52.  34.  32.  34.  35.  36.  40.  60.  35.  47.  49.
  44.  51.  45.  49.  34.  46.  41.  45.  81.  44.  33.]
First 10 rows of predictions:
       Actual  Predicted
6152    42.0       42.0
6030   101.0      122.0
6169    51.0       40.0
6110    53.0       55.0
6082    53.0       56.0
6167    54.0       60.0
6069    61.0       64.0
6029   127.0      122.0
6126    48.0       53.0
6125    46.0       36.0


,Actual,Predicted
6152,42.0,42.0
6030,101.0,122.0
6169,51.0,40.0
6110,53.0,55.0
6082,53.0,56.0
6167,54.0,60.0
6069,61.0,64.0
6029,127.0,122.0
6126,48.0,53.0
6125,46.0,36.0


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize an empty list to store the predictions DataFrames
predictions_dfs = []

# Get the unique jurisdictions
jurisdictions = data['Jurisdiction of Occurrence'].unique()

# Get the columns of interest
columns_of_interest = ['Septicemia (A40-A41)','Malignant neoplasms (C00-C97)','Diabetes mellitus (E10-E14)','Alzheimer disease (G30)','Influenza and pneumonia (J09-J18)', 'Chronic lower respiratory diseases (J40-J47)',
                       "Other diseases of respiratory system (J00-J06,J30-J39,J67,J70-J98)","Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)","Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)",
                       "Diseases of heart (I00-I09,I11,I13,I20-I51)",'Cerebrovascular diseases (I60-I69)',"COVID-19 (U071, Multiple Cause of Death)","COVID-19 (U071, Underlying Cause of Death)" ]

# Loop through the jurisdictions
for jurisdiction in jurisdictions:
    # Filter the data
    filtered_data = data[data['Jurisdiction of Occurrence'] == jurisdiction]

    # Loop through the columns of interest
    for column in columns_of_interest:
        # Preprocessing steps
        # Convert the 'Data As Of' column to datetime objects
        filtered_data['Data As Of'] = pd.to_datetime(filtered_data['Data As Of'], format='%m/%d/%Y')


        filtered_data = filtered_data.drop('Jurisdiction of Occurrence', axis=1)
        filtered_data = filtered_data.drop('Week Ending Date', axis=1)
        filtered_data.drop(filtered_data.columns[20:36], axis=1, inplace=True)
        filtered_data = filtered_data.drop('flag_allcause', axis=1)
        filtered_data = filtered_data.drop('flag_natcause', axis=1)
       
       

# Extract year, month, and day as separate columns
        filtered_data['Year'] = filtered_data['Data As Of'].dt.year
        filtered_data['Month'] = filtered_data['Data As Of'].dt.month
        filtered_data['Day'] = filtered_data['Data As Of'].dt.day

# Drop the original 'Date' column if it's no longer needed
        filtered_data = filtered_data.drop('Data As Of', axis=1)

        # Fill missing values
        filtered_data = filtered_data.fillna(0)

        # Split the data into features (X) and target (y)
        X = filtered_data.drop(column, axis=1)
        y = filtered_data[column]

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train the model
        dt_regressor = DecisionTreeRegressor(random_state=42)
        dt_regressor.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = dt_regressor.predict(X_test)

        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Create a DataFrame with the actual and predicted values
        predictions_df = pd.DataFrame({
            'Actual': y_test,
            'Predicted': y_pred,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        })

        # Add the jurisdiction name and column name to the DataFrame
        predictions_df['Jurisdiction'] = jurisdiction
        predictions_df['Column'] = column

        # Append the DataFrame to the list
        predictions_dfs.append(predictions_df)
        filtered_data = data[data['Jurisdiction of Occurrence'] == jurisdiction].reset_index(drop=True)


# Concatenate the predictions DataFrames into a single DataFrame
combined_predictions_df = pd.concat(predictions_dfs, ignore_index=True)

# Save the DataFrame to a CSV file
combined_predictions_df.to_csv("combined_predictions.csv", index=False)


C:\Users\pepto\AppData\Local\Temp\ipykernel_13928\4145502949.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Data As Of'] = pd.to_datetime(filtered_data['Data As Of'], format='%m/%d/%Y')
C:\Users\pepto\AppData\Local\Temp\ipykernel_13928\4145502949.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Data As Of'] = pd.to_datetime(filtered_data['Data As Of'], format='%m/%d/%Y')
C:\Users\pepto\AppData\Local\Temp\ipykernel_13928\4145502949.py:27: SettingWithCopyWarning: 
A val

In [16]:
combined_predictions_df

,Actual,Predicted,MSE,RMSE,MAE,R2,Jurisdiction,Column
0,767.0,730.0,2684.743590,51.814511,40.384615,0.410636,United States,Septicemia (A40-A41)
1,735.0,843.0,2684.743590,51.814511,40.384615,0.410636,United States,Septicemia (A40-A41)
2,966.0,993.0,2684.743590,51.814511,40.384615,0.410636,United States,Septicemia (A40-A41)
3,816.0,815.0,2684.743590,51.814511,40.384615,0.410636,United States,Septicemia (A40-A41)
4,729.0,699.0,2684.743590,51.814511,40.384615,0.410636,United States,Septicemia (A40-A41)
...,...,...,...,...,...,...,...,...
27373,36.0,32.0,22.820513,4.777082,2.615385,0.932451,Puerto Rico,"COVID-19 (U071, Underlying Cause of Death)"
27374,0.0,0.0,22.820513,4.777082,2.615385,0.932451,Puerto Rico,"COVID-19 (U071, Underlying Cause of Death)"
27375,0.0,0.0,22.820513,4.777082,2.615385,0.932451,Puerto Rico,"COVID-19 (U071, Underlying Cause of Death)"
27376,0.0,0.0,22.820513,4.777082,2.615385,0.932451,Puerto Rico,"COVID-19 (U071, Underlying Cause of Death)"


In [17]:
# Group by "Jurisdiction" and "Column" and sum the "Predicted" values
df = combined_predictions_df.groupby(['Jurisdiction', 'Column'], as_index=False)['Predicted'].sum()

# Pivot the DataFrame
df_pivot = df.pivot(index='Jurisdiction', columns='Column', values='Predicted')

# Reset the index to make it a regular column
df_pivot = df_pivot.reset_index()

In [18]:
df_pivot

Column,Jurisdiction,Alzheimer disease (G30),"COVID-19 (U071, Multiple Cause of Death)","COVID-19 (U071, Underlying Cause of Death)",Cerebrovascular diseases (I60-I69),Chronic lower respiratory diseases (J40-J47),Diabetes mellitus (E10-E14),"Diseases of heart (I00-I09,I11,I13,I20-I51)",Influenza and pneumonia (J09-J18),Malignant neoplasms (C00-C97),"Nephritis, nephrotic syndrome and nephrosis (N00-N07,N17-N19,N25-N27)","Other diseases of respiratory system (J00-J06,J30-J39,J67,J70-J98)",Septicemia (A40-A41),"Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified (R00-R99)"
0,Alabama,2026.0,3421.0,3022.0,2453.0,2329.0,1062.0,10736.0,693.0,7542.0,934.0,755.0,819.0,821.0
1,Alaska,0.0,239.0,155.0,0.0,0.0,0.0,697.0,0.0,760.0,0.0,0.0,0.0,0.0
2,Arizona,2154.0,4852.0,4560.0,2283.0,2722.0,1827.0,10948.0,756.0,9681.0,668.0,885.0,226.0,854.0
3,Arkansas,1183.0,1842.0,1589.0,1266.0,1797.0,1049.0,6191.0,452.0,4825.0,542.0,267.0,371.0,66.0
4,California,12466.0,16994.0,15837.0,13498.0,8945.0,8691.0,48690.0,3836.0,45124.0,3382.0,2271.0,1249.0,1947.0
5,Colorado,1426.0,2977.0,2511.0,1657.0,1842.0,824.0,6310.0,147.0,6183.0,253.0,477.0,242.0,120.0
6,Connecticut,809.0,3171.0,2885.0,1130.0,903.0,513.0,5170.0,232.0,4918.0,404.0,419.0,207.0,91.0
7,Delaware,219.0,677.0,461.0,616.0,270.0,49.0,1672.0,10.0,1648.0,0.0,0.0,0.0,33.0
8,District of Columbia,0.0,359.0,385.0,79.0,0.0,0.0,1094.0,0.0,926.0,0.0,0.0,0.0,30.0
9,Florida,4935.0,12995.0,10731.0,12040.0,8292.0,5706.0,37901.0,2410.0,35139.0,2622.0,2133.0,2097.0,3050.0


In [19]:
# Save the DataFrame to a CSV file
df_pivot.to_csv("df_pivot.csv", index=False)